In [1]:
from gurobipy import *  # import the optimize solver Gurobi

In [2]:
number_of_product = 2 # Set the index for number of product(1, 2)
number_of_shipping_routes = 2 # Set the index for number of shipping routes(1, 2)
m = Model() # Import and create the model

Using license file C:\Users\victo\gurobi.lic
Academic license - for non-commercial use only


In [3]:
# Set the input Parameter: 
shipping_hour = [[3, 3], [2, 3]] # Shipping days required to ship each type of product on each route
labor_hour = [[15, 15.5], [16, 14]]  # Labor hours required to ship each type of product on each route  
unit_cost = [[165, 135], [180, 145]]   # Unit cost to ship each type of product on each route 
demand = [10, 8]  # Demand to ship each type of product
each_product = [720, 720]   # hours available on shipping each product

In [4]:
# Set the Variable list: Number of each type of product shipped on each route
# Set the Variable nx to integer number
nx = [[0,0], [0,0]]
for i in range(number_of_shipping_routes):
    for j in range(number_of_product):
        nx[i][j] = m.addVar(vtype=GRB.INTEGER, name='nx{}'.format(str(i + 1) + str(j + 1)))

In [5]:
# Set the Minimize Obijective: Net Cost
m.setObjective(quicksum([sum([unit_cost[i][j] * nx[i][j] for j in range(number_of_product)]) for i in range(number_of_shipping_routes)]) ,  GRB.MINIMIZE)

In [6]:
# Set Non Negative decision variable
for i in range(number_of_shipping_routes):
     for j in range(number_of_product):
            c1 = m.addConstr(nx[i][j]  >= 0)

In [7]:
# Shippiing routes constraint
c2 = []
for i in range(number_of_shipping_routes):
    c2.append(m.addConstr(sum([shipping_hour[i][j]*nx[i][j] for j in range(number_of_product)]) <= each_product[i]))

In [8]:
# Labor hours constraint
c3 = m.addConstr( quicksum([sum([labor_hour[i][j]*nx[i][j] for j in range(number_of_product)]) for i in range(number_of_shipping_routes)]) <= 400)

In [9]:
# Production Demand constraint
c4 = []
for j in range(number_of_product):
    c4.append(m.addConstr(sum([nx[i][j] for i in range(number_of_shipping_routes)]) >= demand[j]))

In [10]:
# Demand to produce each type of project on each route(at least half of the product 1 requirement must be shipped on route 2, and at least half of the product 2 requirement must be shipped on route 1)
c5 = m.addConstr(nx[0][1] >= 1/2 * demand[1])
c6 = m.addConstr(nx[1][0] >= 1/2 * demand[0])

In [11]:
# Run the optimize solver
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 11 rows, 4 columns and 18 nonzeros
Model fingerprint: 0x814a5bd8
Variable types: 0 continuous, 4 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 7e+02]
Presolve removed 11 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 2805 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.805000000000e+03, best bound 2.805000000000e+03, gap 0.0000%


In [12]:
# Get the Optimal Solution for X
m.printAttr('X')


    Variable            X 
-------------------------
        nx11            5 
        nx12            8 
        nx21            5 


In [13]:
# Get the Optimal Objective Value
m.ObjVal

2805.0